<a href="https://colab.research.google.com/github/sydney-smith/NMA_project/blob/main/sid_entropy3_NMA_analyze_LFP(Spectra).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
#@title Install external software packages
!pip install neurodsp
!pip install fooof

In [23]:
#@title import analysis packages
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from neurodsp import spectral
from fooof import FOOOFGroup
from fooof import analysis, utils

In [24]:
#@title Import matplotlib and set defaults
from matplotlib import rcParams 
from matplotlib import pyplot as plt
rcParams['figure.figsize'] = [20, 4]
rcParams['font.size'] =15
rcParams['axes.spines.top'] = False
rcParams['axes.spines.right'] = False
rcParams['figure.autolayout'] = True

In [25]:
#@title Data retrieval (LFP)

import os, requests

fname = ['steinmetz_st.npz']
fname.append('steinmetz_wav.npz')
fname.append('steinmetz_lfp.npz')

url = ["https://osf.io/4bjns/download"]
url.append("https://osf.io/ugm9v/download")
url.append("https://osf.io/kx3v9/download")

for j in range(len(url)):
  if not os.path.isfile(fname[j]):
    try:
      r = requests.get(url[j])
    except requests.ConnectionError:
      print("!!! Failed to download data !!!")
    else:
      if r.status_code != requests.codes.ok:
        print("!!! Failed to download data !!!")
      else:
        with open(fname[j], "wb") as fid:
          fid.write(r.content)

In [26]:
#@title Data retrieval (behavior)
import os, requests

fname = []
for j in range(3):
  fname.append('steinmetz_part%d.npz'%j)
url = ["https://osf.io/agvxh/download"]
url.append("https://osf.io/uv3mw/download")
url.append("https://osf.io/ehmw2/download")

for j in range(len(url)):
  if not os.path.isfile(fname[j]):
    try:
      r = requests.get(url[j])
    except requests.ConnectionError:
      print("!!! Failed to download data !!!")
    else:
      if r.status_code != requests.codes.ok:
        print("!!! Failed to download data !!!")
      else:
        with open(fname[j], "wb") as fid:
          fid.write(r.content)

In [27]:
#@title Data loading

dat_LFP = np.load('steinmetz_lfp.npz', allow_pickle=True)['dat']
# dat_WAV = np.load('steinmetz_wav.npz', allow_pickle=True)['dat']
# dat_ST = np.load('steinmetz_st.npz', allow_pickle=True)['dat']

alldat = np.array([])
for j in range(len(fname)):
  alldat = np.hstack((alldat, np.load('steinmetz_part%d.npz'%j, allow_pickle=True)['dat']))

# select just one of the recordings here. 11 is nice because it has some neurons in vis ctx. 
dat = dat_LFP[11]
print(dat.keys())
dat_k = alldat[11]
print(dat_k.keys())
# dat = dat_WAV[11]
# print(dat.keys())
# dat = dat_ST[11]
# print(dat.keys())

dict_keys(['lfp', 'lfp_passive', 'brain_area_lfp'])
dict_keys(['spks', 'wheel', 'pupil', 'response', 'response_time', 'bin_size', 'stim_onset', 'contrast_right', 'contrast_left', 'brain_area', 'feedback_time', 'feedback_type', 'gocue', 'mouse_name', 'date_exp', 'trough_to_peak', 'active_trials', 'contrast_left_passive', 'contrast_right_passive', 'spks_passive', 'pupil_passive', 'wheel_passive', 'prev_reward', 'ccf', 'ccf_axes', 'cellid_orig', 'reaction_time', 'face', 'face_passive', 'licks', 'licks_passive'])


In [ ]:
# groupings of brain regions
regions = ["vis ctx", "thal", "hipp", "other ctx", "midbrain", "basal ganglia", "cortical subplate", "other"]
brain_groups = [["VISa", "VISam", "VISl", "VISp", "VISpm", "VISrl"], # visual cortex
                ["CL", "LD", "LGd", "LH", "LP", "MD", "MG", "PO", "POL", "PT", "RT", "SPF", "TH", "VAL", "VPL", "VPM"], # thalamus
                ["CA", "CA1", "CA2", "CA3", "DG", "SUB", "POST"], # hippocampal
                ["ACA", "AUD", "COA", "DP", "ILA", "MOp", "MOs", "OLF", "ORB", "ORBm", "PIR", "PL", "SSp", "SSs", "RSP"," TT"], # non-visual cortex
                ["APN", "IC", "MB", "MRN", "NB", "PAG", "RN", "SCs", "SCm", "SCig", "SCsg", "ZI"], # midbrain
                ["ACB", "CP", "GPe", "LS", "LSc", "LSr", "MS", "OT", "SNr", "SI"], # basal ganglia 
                ["BLA", "BMA", "EP", "EPd", "MEA"] # cortical subplate
                ]

In [ ]:
#@title Helper functions

def get_rec_dat(alldat, dat_LFP, i):
  "gets information for single recording session"

  mouse_name = alldat[i]['mouse_name']
  n_shanks = dat_LFP[i]['lfp'].shape[0]
  n_trials = dat_LFP[i]['lfp'].shape[1]
  n_samples = dat_LFP[i]['lfp'].shape[2]
  sfreq = n_samples/2.5 # 100 Hz sampling rate (nyquist @ 50 Hz)

  return mouse_name, n_shanks, n_trials, n_samples, sfreq

def get_trial_dat(alldat, dat_LFP, i, shank, trial):
  "gets data from single trial"

  sig = dat_LFP[i]['lfp'][shank][trial]
  contrast_left = alldat[i]['contrast_left'][trial]
  contrast_right = alldat[i]['contrast_right'][trial]
  response = alldat[i]['response'][trial]
  response_time = alldat[i]['response_time'][trial]

  return sig, contrast_left, contrast_right, response, response_time

def spec_param_features(freqs, spectrum, fm):
  "returns features of parameterized power spectrum"

  fm.fit(freqs, spectrum, freq_range=[1,50])
  ap_exp = fm.aperiodic_params_[1]
  offset = fm.aperiodic_params_[0]
  gamma = analysis.get_band_peak_fm(fm, [30,50])
  gamma_cf = gamma[0]
  gamma_pow = gamma[1]

  return ap_exp, offset, gamma_cf, gamma_pow


Spectral Parameter

In [37]:
fg = FOOOFGroup(peak_width_limits=(2,6))

recordings = np.array([])
mouse_names = np.array([])
n_recording = np.array([])
shanks = np.array([])
probe_locs = np.array([])
trials = np.array([])
contrast_lefts = np.array([])
contrast_rights = np.array([])
contrast_diffs = np.array([])
responses = np.array([])
response_times = np.array([])
ap_exps = np.array([])
offsets = np.array([])
theta_cfs = np.array([])
theta_pows = np.array([])
theta_bands = np.array([])
beta_cfs = np.array([])
beta_pows = np.array([])
beta_bands = np.array([])
gamma_cfs = np.array([])
gamma_pows = np.array([])
gamma_bands = np.array([])
!pip install antropy 

for i in range(0, 1): #alldat.shape[0]
  
  mouse_name, n_shanks, n_trials, n_samples, sfreq = get_rec_dat(alldat, dat_LFP, i)
  print('loading recording '+ str(i) +', mouse: '+ mouse_name)

  for shank in range(0,n_shanks):

    probe_loc = dat_LFP[i]['brain_area_lfp'][shank]

    perm_entropy_trials = []
    spectral_entropy_trials = []
    svd_entropy_trials = []
    app_entropy_trials = []
    sample_entropy_trials = []

    for trial in range(0,n_trials):

      sig, contrast_left, contrast_right, response, response_time = get_trial_dat(alldat, dat_LFP, i, shank, trial)
      perm_entropy = ant.perm_entropy(sig[25:125])
      perm_entropy_trials.append(perm_entropy)
      # spectral_entropy = ant.spectral_entropy(sig[25:125], sf=100, method='welch', normalize=True)
      # spectral_entropy_trials.append(spectral_entropy)
      svd_entropy = ant.svd_entropy(sig[25:125], normalize=True)
      svd_entropy_trials.append(svd_entropy)
      app_entropy = ant.app_entropy(sig[25:125])
      app_entropy_trials.append(app_entropy)
      sample_entropy = ant.sample_entropy(sig[25:125])
      sample_entropy_trials.append(sample_entropy)

    # sig = dat_LFP[i]['lfp'][shank]
      # print (sig[25:125])
      # freqs, spectra = spectral.compute_spectrum_welch(sig[25:125], sfreq, avg_type='median', nperseg=sfreq, noverlap=sfreq/1.5)
      # break

#     fg.fit(freqs, spectra, freq_range=[1,50])
#     ap_exp = fg.get_params('aperiodic_params', 'exponent')
#     offset = fg.get_params('aperiodic_params', 'offset')
#     theta = analysis.get_band_peak_fg(fg, [3,8]) #tuples of cf, pwr
#     beta = analysis.get_band_peak_fg(fg, [13,30]) #tuples of cf, pwr
#     gamma = analysis.get_band_peak_fg(fg, [30,50]) #tuples of cf, pwr
#     theta_ext = utils.trim_spectrum(freqs, spectra, [8,13])
#     theta_band = np.trapz(theta_ext[1], theta_ext[0])
#     beta_ext = utils.trim_spectrum(freqs, spectra, [13,30])
#     beta_band = np.trapz(beta_ext[1], beta_ext[0])
#     gamma_ext = utils.trim_spectrum(freqs, spectra, [30,50])
#     gamma_band = np.trapz(gamma_ext[1], gamma_ext[0])

    shanks = np.concatenate([shanks, np.tile(shank, n_trials)], axis=None)
    probe_locs = np.concatenate([probe_locs, np.tile(probe_loc, n_trials)], axis=None)
    trials = np.concatenate([trials, np.arange(0,n_trials)], axis=None)

    contrast_left = alldat[i]['contrast_left']
    contrast_right = alldat[i]['contrast_right']
    contrast_lefts = np.concatenate([contrast_lefts, contrast_left], axis=None)
    contrast_rights = np.concatenate([contrast_rights, contrast_right], axis=None)
    contrast_diffs = np.concatenate([contrast_diffs, contrast_left-contrast_right], axis=None)
    responses = np.concatenate([responses, alldat[i]['response']], axis=None)
    response_times = np.concatenate([response_times, alldat[i]['response_time']], axis=None)

    ap_exps = np.concatenate([ap_exps, ap_exp], axis=None)
    offsets = np.concatenate([offsets, offset], axis=None)

#     gamma_pows = np.concatenate([gamma_pows, gamma[:,1]], axis=None)
#     gamma_bands = np.concatenate([gamma_bands, gamma_band], axis=None)

    print(np.mean(perm_entropy_trials))
    # print(np.mean(spectral_entropy_trials))
    print(np.mean(svd_entropy_trials))
    print(np.mean(app_entropy_trials))
    print(np.mean(sample_entropy_trials)) 

    perm_entropy_mean = np.mean(perm_entropy_trials)   
    svd_entropy_mean = np.mean(svd_entropy_trials)   
    app_entropy_mean = np.mean(app_entropy_trials)   
    sample_entropy_mean = np.mean(sample_entropy_trials)   

    break
  break    
#     theta_cfs = np.concatenate([theta_cfs, theta[:,0]], axis=None)
#     theta_pows = np.concatenate([theta_pows, theta[:,1]], axis=None)
#     theta_bands = np.concatenate([theta_bands, theta_band], axis=None)
#     beta_cfs = np.concatenate([beta_cfs, beta[:,0]], axis=None)
#     beta_pows = np.concatenate([beta_pows, beta[:,1]], axis=None)
#     beta_bands = np.concatenate([beta_bands, beta_band], axis=None)
#     gamma_cfs = np.concatenate([gamma_cfs, gamma[:,0]], axis=None)
#     gamma_pows = np.concatenate([gamma_pows, gamma[:,1]], axis=None)
#     gamma_bands = np.concatenate([gamma_bands, gamma_band], axis=None)

#   # recording
#   recordings = np.concatenate([recordings, np.tile(i,n_shanks*n_trials)], axis=None)
#   # mouse name
#   mouse_names = np.concatenate([mouse_names, np.tile(mouse_name,n_shanks*n_trials)], axis=None)


# keys = ['recording', 'mouse_name', 'shank', 'brain_area', 'trial', 'contrast_left',
#         'contrast_right', 'contrast_diff', 'response', 'response_time', 'exponent',
#         'offset', 'theta_cf', 'theta_pow', 'theta_band', 'beta_cf', 'beta_pow', 'beta_band',
#         'gamma_cf', 'gamma_pow', 'gamma_band']
# values = [recordings, mouse_names, shanks, probe_locs, trials, contrast_lefts,
#           contrast_rights, contrast_diffs, responses, response_times, ap_exps, 
#           offsets, theta_cfs, theta_pows, theta_bands, beta_cfs, beta_pows, beta_bands,
#           gamma_cfs, gamma_pows, gamma_bands]
# exp_dict = dict(zip(keys, values))



#     for trial in range(0,n_trials):

#       sig, contrast_left, contrast_right, response, response_time = get_trial_dat(alldat, dat_LFP, i, shank, trial)
#       freqs, spectrum = spectral.compute_spectrum_welch(sig[25:125], sfreq, avg_type='median', nperseg=sfreq, noverlap=sfreq/1.5) #wavelet instead for such a short time window?
#       ap_exp, offset, gamma_cf, gamma_pow = spec_param_features(freqs, spectrum, fm)

#       mouse_names.append(mouse_name)
#       shanks.append(shank)
#       probe_locs.append(probe_loc)
#       trials.append(trial)
#       contrast_lefts.append(contrast_left)
#       contrast_rights.append(contrast_right)
#       responses.append(response)
#       response_times.append(response_times)
#       ap_exps.append(ap_exp)
#       offsets.append(offset)
#       # beta
#       gamma_cfs.append(gamma_cf)
#       gamma_pows.append(gamma_pow)

# keys = ['mouse_name', 'shank', 'brain_area', 'trial', 'contrast_left', 'contrast_right', 'response', 'response_time', 'exponent', 'offset', 'gamma_cf', 'gamma_pow']
# lists = [mouse_names, shanks, probe_locs, trials, contrast_lefts, contrast_rights, responses, response_times, ap_exps, offsets, gamma_cfs, gamma_pows]

# exp_dict = dict(zip(keys, lists))

# df = pd.DataFrame(exp_dict)


loading recording 0, mouse: Cori
2.477440395074625
0.791918043374138
0.6835391972598013
1.6064648410930726


In [ ]:
for key in exp_dict:
  print(key, exp_dict[key].shape)

recording (1498,)
mouse_name (1498,)
shank (1498,)
brain_area (1498,)
trial (1498,)
contrast_left (1498,)
contrast_right (1498,)
contrast_diff (1498,)
response (1498,)
response_time (1498,)
exponent (1498,)
offset (1498,)
theta_cf (1498,)
theta_pow (1498,)
theta_band (1498,)
beta_cf (1498,)
beta_pow (1498,)
beta_band (1498,)
gamma_cf (1498,)
gamma_pow (1498,)
gamma_band (1498,)


In [ ]:
df #dataframe for 1 recording (I'll complete the loop and store the output as a CSV so it can be imported later for model fitting without running this analysis)

,recording,mouse_name,shank,brain_area,trial,contrast_left,contrast_right,contrast_diff,response,response_time,exponent,offset,theta_cf,theta_pow,theta_band,beta_cf,beta_pow,beta_band,gamma_cf,gamma_pow,gamma_band
0,0.0,Cori,0.0,ACA,0.0,1.00,0.0,1.00,1.0,1.150204,2.412230,2.664121,NaN,NaN,9.831546,NaN,NaN,10.620027,36.095439,2.014865,18.387417
1,0.0,Cori,0.0,ACA,1.0,0.00,0.5,-0.50,-1.0,1.399503,2.107879,2.144954,NaN,NaN,11.997439,19.000000,0.938180,15.890682,45.826713,1.538463,10.291032
2,0.0,Cori,0.0,ACA,2.0,1.00,0.5,0.50,1.0,0.949291,1.586710,1.446829,3.226824,1.225611,6.192864,21.057067,1.104569,9.401584,38.460663,0.924613,6.324518
3,0.0,Cori,0.0,ACA,3.0,0.00,0.0,0.00,0.0,2.266802,1.621517,1.252535,6.153481,0.762158,6.245720,NaN,NaN,5.735772,46.344240,1.335879,6.463771
4,0.0,Cori,0.0,ACA,4.0,0.50,1.0,-0.50,1.0,0.816776,1.524745,1.212975,NaN,NaN,10.501702,26.125270,1.304335,17.662793,45.406778,1.445720,10.705526
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1493,0.0,Cori,6.0,VISp,209.0,0.25,1.0,-0.75,1.0,1.453137,0.602434,0.300598,6.493110,1.688923,24.784042,29.439942,1.016407,16.433683,NaN,NaN,8.210993
1494,0.0,Cori,6.0,VISp,210.0,0.25,1.0,-0.75,1.0,1.220359,1.419854,1.310525,6.027076,0.864218,19.192282,14.000000,0.774499,13.505859,35.515767,1.301021,8.769719
1495,0.0,Cori,6.0,VISp,211.0,0.25,1.0,-0.75,0.0,2.306214,0.641058,-0.105132,6.069379,1.609233,5.915201,25.945036,1.246247,10.611638,38.737092,1.267290,8.728672
1496,0.0,Cori,6.0,VISp,212.0,0.25,1.0,-0.75,0.0,2.277861,1.166195,0.814729,5.621821,1.587544,16.438089,NaN,NaN,4.514922,38.584163,1.164875,8.151036
